In [59]:
import pandas as pd
import numpy as np

Reference: https://dhiraj-p-rai.medium.com/logistic-regression-in-spark-ml-8a95b5f5434c

In [60]:
import pyspark
from pyspark.sql.functions import col
from pyspark.sql.functions import when

In [61]:
spark = pyspark.sql.SparkSession.builder.master('local[1]').appName('H1B').getOrCreate()
training_data = spark.read.csv('train.csv', header = True, inferSchema = True)

In [62]:
test_data = spark.read.csv('test.csv', header = True, inferSchema = True)

21/10/31 19:38:01 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 129102 ms exceeds timeout 120000 ms
21/10/31 19:38:01 WARN SparkContext: Killing executors is not supported by current scheduler.


In [32]:
cols = training_data.columns
cols.remove('EMPLOYER_NAME')

In [33]:
training_data = training_data.drop(col('EMPLOYER_NAME'))
training_data = training_data.withColumn("CASE_DURATION",col("CASE_DURATION").cast("int"))
training_data = training_data.withColumn("FULL_TIME_POSITION",col("FULL_TIME_POSITION").cast("int"))
training_data.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- AK1: integer (nullable = true)
 |-- AL2: integer (nullable = true)
 |-- AR3: integer (nullable = true)
 |-- AZ4: integer (nullable = true)
 |-- CA5: integer (nullable = true)
 |-- CO6: integer (nullable = true)
 |-- CT7: integer (nullable = true)
 |-- DC8: integer (nullable = true)
 |-- DE9: integer (nullable = true)
 |-- FL10: integer (nullable = true)
 |-- FM11: integer (nullable = true)
 |-- GA12: integer (nullable = true)
 |-- GU13: integer (nullable = true)
 |-- HI14: integer (nullable = true)
 |-- IA15: integer (nullable = true)
 |-- ID16: integer (nullable = true)
 |-- IL17: integer (nullable = true)
 |-- IN18: integer (nullable = true)
 |-- KS19: integer (nullable = true)
 |-- KY20: integer (nullable = true)
 |-- LA21: integer (nullable = true)
 |-- MA22: integer (nullable = true)
 |-- MD23: integer (nullable = true)
 |-- ME24: integer (nullable = true)
 |-- MH: integer (nullable = true)
 |-- MI26: integer (nullable = true)
 |-- MN2

In [34]:
from pyspark.ml.feature import VectorAssembler
try:
  cols.remove('CASE_STATUS')
except:
  pass
cols.remove('_c0')
training_data = training_data.drop(col('_c0'))
print(cols)
from pyspark.ml.linalg import SparseVector, DenseVector 
assembler = VectorAssembler(inputCols=cols,outputCol="features")

['AK1', 'AL2', 'AR3', 'AZ4', 'CA5', 'CO6', 'CT7', 'DC8', 'DE9', 'FL10', 'FM11', 'GA12', 'GU13', 'HI14', 'IA15', 'ID16', 'IL17', 'IN18', 'KS19', 'KY20', 'LA21', 'MA22', 'MD23', 'ME24', 'MH', 'MI26', 'MN27', 'MO28', 'MP29', 'MS30', 'MT31', 'NC32', 'ND33', 'NE34', 'NH35', 'NJ36', 'NM37', 'NV38', 'NY39', 'OH40', 'OK41', 'OR42', 'PA43', 'PR44', 'PW45', 'RI46', 'SC47', 'SD48', 'TN49', 'TX50', 'UT51', 'VA52', 'VI53', 'VT54', 'WA55', 'WI56', 'WV57', 'WY58', '11-1', '11-2', '11-3', '11-9', '13-1', '13-2', '15-1', '15-2', '15-9', '16-1', '17-1', '17-2', '17-3', '19-1', '19-2', '19-3', '19-4', '20-2', '21-1', '21-2', '23-1', '23-2', '25-1', '25-2', '25-3', '25-4', '25-7', '25-9', '26-1', '27-1', '27-2', '27-3', '27-4', '29-1', '29-2', '29-9', '31-1', '31-2', '31-9', '33-1', '33-2', '33-3', '33-9', '35-1', '35-2', '35-3', '35-9', '37-1', '37-2', '37-3', '39-1', '39-2', '39-3', '39-4', '39-5', '39-6', '39-7', '39-9', '40-9', '41-1', '41-2', '41-3', '41-4', '41-9', '43-1', '43-3', '43-4', '43-5', '4

In [35]:
training_data = training_data.withColumn("CASE_STATUS",
                                         when(training_data.CASE_STATUS == 'CERTIFIED',1)
                                         .otherwise(when(training_data.CASE_STATUS == 'DENIED',2)
                                         .otherwise(when(training_data.CASE_STATUS == 'WITHDRAWN',3)
                                         .otherwise(4))))

In [57]:
training_data.dropna(subset=cols)

DataFrame[AK1: int, AL2: int, AR3: int, AZ4: int, CA5: int, CO6: int, CT7: int, DC8: int, DE9: int, FL10: int, FM11: int, GA12: int, GU13: int, HI14: int, IA15: int, ID16: int, IL17: int, IN18: int, KS19: int, KY20: int, LA21: int, MA22: int, MD23: int, ME24: int, MH: int, MI26: int, MN27: int, MO28: int, MP29: int, MS30: int, MT31: int, NC32: int, ND33: int, NE34: int, NH35: int, NJ36: int, NM37: int, NV38: int, NY39: int, OH40: int, OK41: int, OR42: int, PA43: int, PR44: int, PW45: int, RI46: int, SC47: int, SD48: int, TN49: int, TX50: int, UT51: int, VA52: int, VI53: int, VT54: int, WA55: int, WI56: int, WV57: int, WY58: int, 11-1: int, 11-2: int, 11-3: int, 11-9: int, 13-1: int, 13-2: int, 15-1: int, 15-2: int, 15-9: int, 16-1: int, 17-1: int, 17-2: int, 17-3: int, 19-1: int, 19-2: int, 19-3: int, 19-4: int, 20-2: int, 21-1: int, 21-2: int, 23-1: int, 23-2: int, 25-1: int, 25-2: int, 25-3: int, 25-4: int, 25-7: int, 25-9: int, 26-1: int, 27-1: int, 27-2: int, 27-3: int, 27-4: int, 

In [53]:
training_data = assembler.transform(training_data)
from pyspark.ml.classification import LogisticRegression
training_data.select('features').show()


+--------------------+
|            features|
+--------------------+
|(756,[42,64,159,2...|
|(756,[4,121,159,1...|
|(756,[49,92,159,2...|
|(756,[7,64,159,17...|
|(756,[38,62,159,2...|
|(756,[4,64,159,16...|
|(756,[4,60,159,16...|
|(756,[4,64,159,16...|
|(756,[49,64,159,1...|
|(756,[39,81,159,1...|
|(756,[9,62,159,17...|
|(756,[54,65,159,2...|
|(756,[4,64,159,16...|
|(756,[4,65,159,21...|
|(756,[42,64,159,1...|
|(756,[26,64,159,1...|
|(756,[49,59,159,2...|
|(756,[11,64,159,2...|
|(756,[33,64,159,1...|
|(756,[31,64,159,2...|
+--------------------+
only showing top 20 rows



21/10/30 18:12:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: AK, AL, AR, AZ, CA, CO, CT, DC, DE, FL, FM, GA, GU, HI, IA, ID, IL, IN, KS, KY, LA, MA, MD, ME, MH, MI, MN, MO, MP, MS, MT, NC, ND, NE, NH, NJ, NM, NV, NY, OH, OK, OR, PA, PR, PW, RI, SC, SD, TN, TX, UT, VA, VI, VT, WA, WI, WV, WY, 11-1, 11-2, 11-3, 11-9, 13-1, 13-2, 15-1, 15-2, 15-9, 16-1, 17-1, 17-2, 17-3, 19-1, 19-2, 19-3, 19-4, 20-2, 21-1, 21-2, 23-1, 23-2, 25-1, 25-2, 25-3, 25-4, 25-7, 25-9, 26-1, 27-1, 27-2, 27-3, 27-4, 29-1, 29-2, 29-9, 31-1, 31-2, 31-9, 33-1, 33-2, 33-3, 33-9, 35-1, 35-2, 35-3, 35-9, 37-1, 37-2, 37-3, 39-1, 39-2, 39-3, 39-4, 39-5, 39-6, 39-7, 39-9, 40-9, 41-1, 41-2, 41-3, 41-4, 41-9, 43-1, 43-3, 43-4, 43-5, 43-6, 43-9, 45-1, 45-2, 45-3, 45-4, 47-1, 47-2, 47-3, 47-4, 47-5, 49-1, 49-2, 49-3, 49-9, 51-1, 51-2, 51-3, 51-4, 51-5, 51-6, 51-7, 51-8, 51-9, 53-1, 53-2, 53-3, 53-5, 53-6, 53-7, 54-1, 71-2, E-3 Australian, H-1B, H-1B1 Chile, H-1B1 Singapore, AK, AL, AR, AS, AZ, CA,

In [54]:
lr = LogisticRegression(labelCol = 'CASE_STATUS',
                        featuresCol = 'features',
                        maxIter=10, regParam=0, elasticNetParam=0.8).setFamily("multinomial")

In [58]:
lr.fit(training_data)

21/10/30 18:15:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: AK, AL, AR, AZ, CA, CO, CT, DC, DE, FL, FM, GA, GU, HI, IA, ID, IL, IN, KS, KY, LA, MA, MD, ME, MH, MI, MN, MO, MP, MS, MT, NC, ND, NE, NH, NJ, NM, NV, NY, OH, OK, OR, PA, PR, PW, RI, SC, SD, TN, TX, UT, VA, VI, VT, WA, WI, WV, WY, 11-1, 11-2, 11-3, 11-9, 13-1, 13-2, 15-1, 15-2, 15-9, 16-1, 17-1, 17-2, 17-3, 19-1, 19-2, 19-3, 19-4, 20-2, 21-1, 21-2, 23-1, 23-2, 25-1, 25-2, 25-3, 25-4, 25-7, 25-9, 26-1, 27-1, 27-2, 27-3, 27-4, 29-1, 29-2, 29-9, 31-1, 31-2, 31-9, 33-1, 33-2, 33-3, 33-9, 35-1, 35-2, 35-3, 35-9, 37-1, 37-2, 37-3, 39-1, 39-2, 39-3, 39-4, 39-5, 39-6, 39-7, 39-9, 40-9, 41-1, 41-2, 41-3, 41-4, 41-9, 43-1, 43-3, 43-4, 43-5, 43-6, 43-9, 45-1, 45-2, 45-3, 45-4, 47-1, 47-2, 47-3, 47-4, 47-5, 49-1, 49-2, 49-3, 49-9, 51-1, 51-2, 51-3, 51-4, 51-5, 51-6, 51-7, 51-8, 51-9, 53-1, 53-2, 53-3, 53-5, 53-6, 53-7, 54-1, 71-2, E-3 Australian, H-1B, H-1B1 Chile, H-1B1 Singapore, AK, AL, AR, AS, AZ, CA,

Py4JJavaError: An error occurred while calling o481.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 30.0 failed 1 times, most recent failure: Lost task 1.0 in stage 30.0 (TID 93) (172.20.17.36 executor driver): org.apache.spark.SparkException: Failed to execute user defined function (VectorAssembler$$Lambda$2870/1102582279: (struct<AK1_double_VectorAssembler_7a847d94a176:double,AL2_double_VectorAssembler_7a847d94a176:double,AR3_double_VectorAssembler_7a847d94a176:double,AZ4_double_VectorAssembler_7a847d94a176:double,CA5_double_VectorAssembler_7a847d94a176:double,CO6_double_VectorAssembler_7a847d94a176:double,CT7_double_VectorAssembler_7a847d94a176:double,DC8_double_VectorAssembler_7a847d94a176:double,DE9_double_VectorAssembler_7a847d94a176:double,FL10_double_VectorAssembler_7a847d94a176:double,FM11_double_VectorAssembler_7a847d94a176:double,GA12_double_VectorAssembler_7a847d94a176:double,GU13_double_VectorAssembler_7a847d94a176:double,HI14_doub

21/10/30 18:15:35 WARN TaskSetManager: Lost task 2.0 in stage 30.0 (TID 94) (172.20.17.36 executor driver): TaskKilled (Stage cancelled)


In [7]:
training_data = training_data.drop(columns=['CASE_STATUS'])

In [8]:
test_data = test_data.drop(columns=['CASE_STATUS'])

In [9]:
try:
    training_data = training_data.drop(columns=['EMPLOYER_NAME','Unnamed: 0'])
except:
    pass
try:
    test_data = test_data.drop(columns=['EMPLOYER_NAME','Unnamed: 0'])
except:
    pass
training_data.head()

,AK,AL,AR,AZ,CA,CO,CT,DC,DE,FL,FM,GA,GU,HI,IA,ID,IL,IN,KS,KY,LA,MA,MD,ME,MH,MI,MN,MO,MP,MS,MT,NC,ND,NE,NH,NJ,NM,NV,NY,OH,...,8121,8122,8123,8129,8131,8132,8133,8134,8139,8141,8211,8221,8231,8322,8399,8418,8621,8711,8742,9199,9211,9221,9231,9241,9251,9261,9271,9281,9986,9999,PREVAILING_WAGE,FULL_TIME_POSITION,total_wage,CASE_DURATION,EMPLOYMENT_DURATION,EMPLOYER_APPLICATION_COUNT,EMPLOYER_CERTIFIED_RATE,EMPLOYER_CERTIFIED-WITHDRAWN_RATE,EMPLOYER_DENIED_RATE,EMPLOYER_WITHDRAWN_RATE
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1336192,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,74963.0,1,74963.0,4,1094.0,36828,0.978386,0.000244,0.011703,0.009667
1265192,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,101650.0,1,101650.0,6,1095.0,138,0.847826,0.130435,0.007246,0.014493
2347539,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48000.0,1,92160000.0,6,1095.0,5,1.000000,0.000000,0.000000,0.000000
1801546,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,72000.0,1,72000.0,6,1096.0,11,0.636364,0.090909,0.181818,0.090909
2401379,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63336.0,1,63336.0,4,1095.0,3,1.000000,0.000000,0.000000,0.000000


In [10]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as qda

In [ ]:
# The training may fail or may cost about 40min.
qda_model = qda()
qda_model.fit(training_data, y_training)


In [ ]:
prediction = qda_model.predict(test_data)


Error: Session cannot generate requests

In [ ]:
np.mean(prediction == y_test)


In [27]:
training_data.head(1)


21/10/30 17:54:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: AK, AL, AR, AZ, CA, CO, CT, DC, DE, FL, FM, GA, GU, HI, IA, ID, IL, IN, KS, KY, LA, MA, MD, ME, MH, MI, MN, MO, MP, MS, MT, NC, ND, NE, NH, NJ, NM, NV, NY, OH, OK, OR, PA, PR, PW, RI, SC, SD, TN, TX, UT, VA, VI, VT, WA, WI, WV, WY, 11-1, 11-2, 11-3, 11-9, 13-1, 13-2, 15-1, 15-2, 15-9, 16-1, 17-1, 17-2, 17-3, 19-1, 19-2, 19-3, 19-4, 20-2, 21-1, 21-2, 23-1, 23-2, 25-1, 25-2, 25-3, 25-4, 25-7, 25-9, 26-1, 27-1, 27-2, 27-3, 27-4, 29-1, 29-2, 29-9, 31-1, 31-2, 31-9, 33-1, 33-2, 33-3, 33-9, 35-1, 35-2, 35-3, 35-9, 37-1, 37-2, 37-3, 39-1, 39-2, 39-3, 39-4, 39-5, 39-6, 39-7, 39-9, 40-9, 41-1, 41-2, 41-3, 41-4, 41-9, 43-1, 43-3, 43-4, 43-5, 43-6, 43-9, 45-1, 45-2, 45-3, 45-4, 47-1, 47-2, 47-3, 47-4, 47-5, 49-1, 49-2, 49-3, 49-9, 51-1, 51-2, 51-3, 51-4, 51-5, 51-6, 51-7, 51-8, 51-9, 53-1, 53-2, 53-3, 53-5, 53-6, 53-7, 54-1, 71-2, E-3 Australian, H-1B, H-1B1 Chile, H-1B1 Singapore, AK, AL, AR, AS, AZ, CA,

[Row(AK1=0, AL2=0, AR3=0, AZ4=0, CA5=0, CO6=0, CT7=0, DC8=0, DE9=0, FL10=0, FM11=0, GA12=0, GU13=0, HI14=0, IA15=0, ID16=0, IL17=0, IN18=0, KS19=0, KY20=0, LA21=0, MA22=0, MD23=0, ME24=0, MH=0, MI26=0, MN27=0, MO28=0, MP29=0, MS30=0, MT31=0, NC32=0, ND33=0, NE34=0, NH35=0, NJ36=0, NM37=0, NV38=0, NY39=0, OH40=0, OK41=0, OR42=0, PA43=1, PR44=0, PW45=0, RI46=0, SC47=0, SD48=0, TN49=0, TX50=0, UT51=0, VA52=0, VI53=0, VT54=0, WA55=0, WI56=0, WV57=0, WY58=0, 11-1=0, 11-2=0, 11-3=0, 11-9=0, 13-1=0, 13-2=0, 15-1=1, 15-2=0, 15-9=0, 16-1=0, 17-1=0, 17-2=0, 17-3=0, 19-1=0, 19-2=0, 19-3=0, 19-4=0, 20-2=0, 21-1=0, 21-2=0, 23-1=0, 23-2=0, 25-1=0, 25-2=0, 25-3=0, 25-4=0, 25-7=0, 25-9=0, 26-1=0, 27-1=0, 27-2=0, 27-3=0, 27-4=0, 29-1=0, 29-2=0, 29-9=0, 31-1=0, 31-2=0, 31-9=0, 33-1=0, 33-2=0, 33-3=0, 33-9=0, 35-1=0, 35-2=0, 35-3=0, 35-9=0, 37-1=0, 37-2=0, 37-3=0, 39-1=0, 39-2=0, 39-3=0, 39-4=0, 39-5=0, 39-6=0, 39-7=0, 39-9=0, 40-9=0, 41-1=0, 41-2=0, 41-3=0, 41-4=0, 41-9=0, 43-1=0, 43-3=0, 43-4=0, 43-5=0